# Lookback Put Options (with different Pricing Models)

Price a Lookback option using different models. Lookback options are valuable hedging tools for financial instruments that require the ability to capitalize on the best market prices over a specific period.  For a definition of the payoff, and the dataclass to construct a lookback put option timetable, see [lookback.py](./src/contracts/lookback.py)

In [1]:
from qablet.heston.mc import HestonMCModel

from data.spx_2005_09_15 import (
    basic_info,
    heston_data,
    localvol_data,
    rbergomi_data,
)
from src.contracts.lookback import lookback_put_timetable
from src.model.localvol import LVMCModel
from src.model.rbergomi import rBergomiMCModel

Contract:

Create a Lookback put option contract fixing time to maturity,T=0.2 with num of monitoring points,m=4

In [2]:
info = basic_info()
prc_dt = info["prc_dt"]
ticker = info["ticker"]

start_date = prc_dt
T = 0.2
num_points = 4

timetable = lookback_put_timetable(ticker, start_date, T, num_points)
print(timetable["events"].to_pandas())

                       time   op  quantity      unit track
0 2005-09-14 00:00:00+00:00  NaN       0.0      INIT      
1 2005-10-02 06:18:00+00:00  NaN       0.0    UPDATE      
2 2005-10-20 12:36:00+00:00  NaN       0.0    UPDATE      
3 2005-11-07 18:54:00+00:00  NaN       0.0    UPDATE      
4 2005-11-26 01:12:00+00:00    +       1.0  LOOKBACK      


Compare Models:

Compare different models, starting with Local Volatility Model.

In [3]:
model = LVMCModel()
price, _ = model.price(timetable, localvol_data())
print(f"Localvol price: {price}")

Localvol price: 34.03527948049825


Heston Model:

In [4]:
model = HestonMCModel()
price, _ = model.price(timetable, heston_data())
print(f"Heston price: {price}")

Heston price: 35.153204400514184


Bergomi Model:

In [5]:
model = rBergomiMCModel()
dataset = rbergomi_data()
dataset["MC"]["PATHS"] = 100_000  # very slow with 100_000, 1/250
dataset["MC"]["TIMESTEP"] = 1 / 100
price, _ = model.price(timetable, dataset)
print(f"rBergomi price: {price}")

rBergomi price: 134.37681693394038


Contract:

Create a Lookback put option contract fixing time to maturity,T=0.5 with num of monitoring points,m=[5,10,20,50,100]

Local Vol:

In [6]:
ticker = "SPX"
start_date = prc_dt
T = 0.5

# List of various monitoring points to analyze
monitoring_points_list = [5, 10, 20, 50, 100]
model = LVMCModel()
lv_data = localvol_data()
# Dictionary to store prices for different monitoring points
prices = {}

# Calculate prices for different numbers of monitoring points
for num_points in monitoring_points_list:
    timetable = lookback_put_timetable(ticker, start_date, T, num_points)
    price, _ = model.price(timetable, lv_data)
    prices[num_points] = price
    print(f"LocalVol price with {num_points} monitoring points: {price}")

LocalVol price with 5 monitoring points: 58.960451961158796
LocalVol price with 10 monitoring points: 64.73760939337866
LocalVol price with 20 monitoring points: 69.05550778317841
LocalVol price with 50 monitoring points: 73.19379170581013
LocalVol price with 100 monitoring points: 75.68009294549402


Heston Model:

In [7]:
ticker = "SPX"
start_date = prc_dt
T = 0.5

# List of various monitoring points to analyze
monitoring_points_list = [5, 10, 20, 50, 100]
model = HestonMCModel()
hs_data = heston_data()
# Dictionary to store prices for different monitoring points
prices = {}

# Calculate prices for different numbers of monitoring points
for num_points in monitoring_points_list:
    timetable = lookback_put_timetable(ticker, start_date, T, num_points)
    price, _ = model.price(timetable, hs_data)
    prices[num_points] = price
    print(f"LocalVol price with {num_points} monitoring points: {price}")

LocalVol price with 5 monitoring points: 56.26101578981621
LocalVol price with 10 monitoring points: 60.68500416033413
LocalVol price with 20 monitoring points: 64.04446914964821
LocalVol price with 50 monitoring points: 67.25761828932377
LocalVol price with 100 monitoring points: 69.35216493175507


rbergomi Model:

In [8]:
ticker = "SPX"
start_date = prc_dt
T = 0.5

# List of various monitoring points to analyze
monitoring_points_list = [5, 10, 20, 50, 100]
model = rBergomiMCModel()
rb_data = rbergomi_data()
rb_data["MC"]["PATHS"] = 20_000  # very slow with 100_000, 1/250
rb_data["MC"]["TIMESTEP"] = 1 / 100
# Dictionary to store prices for different monitoring points
prices = {}

# Calculate prices for different numbers of monitoring points
for num_points in monitoring_points_list:
    timetable = lookback_put_timetable(ticker, start_date, T, num_points)
    price, _ = model.price(timetable, rb_data)
    prices[num_points] = price
    print(f"LocalVol price with {num_points} monitoring points: {price}")

LocalVol price with 5 monitoring points: 93.33586816227653
LocalVol price with 10 monitoring points: 385.20046388074513
LocalVol price with 20 monitoring points: 302.54158000217706
LocalVol price with 50 monitoring points: 1831.9457653675563
LocalVol price with 100 monitoring points: 2027.1520901542478


Note: There is clearly something wrong with the rBergomi Model which need to be investigated.